In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from sklearn.metrics import f1_score
import numpy as np

# Hyperparameter Configuration
batch_size = 64
num_epochs = 10
learning_rate = 0.001

In [6]:
# Preparation of Dataset and DataLoader
import os
import numpy as np
from torch.utils.data import SubsetRandomSampler
from torchvision.datasets import ImageFolder
from torchvision import transforms

# Define paths to data directories
root_dir = "/content/drive/MyDrive/음원 mel-spectrogram 7초짜리 복사본"
train_dir = os.path.join(root_dir, "train")
val_dir = os.path.join(root_dir, "val")

# Define transforms for data preprocessing
transform_train = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

transform_val = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Load the full dataset
dataset = ImageFolder(root=root_dir, transform=transform_train)

# Split the dataset into training and validation sets
num_samples = len(dataset)
indices = np.arange(num_samples)
np.random.shuffle(indices)

split = int(np.floor(0.2 * num_samples))
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

# Create data loaders for training and validation sets
batch_size = 64
num_workers = 4  # number of processes to use for data loading
pin_memory = True  # whether to use pinned memory for GPU acceleration
print("Creating data loaders...")
train_dataloader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler, num_workers=num_workers, pin_memory=pin_memory)
val_dataloader = DataLoader(dataset, batch_size=batch_size, sampler=val_sampler, num_workers=num_workers, pin_memory=pin_memory)
print("Data loaders created.")

In [1]:
# Model Configuration
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.fc1 = nn.Linear(in_features=64 * 30 * 30, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=4)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

NameError: name 'nn' is not defined

In [ ]:
# Model Creation and Optimization Function Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Define the path where to save the checkpoint
checkpoint_dir = "/content/drive/MyDrive/checkpoints/7초짜리"
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
best_accuracy = 0.0
best_f1 = 0.0

In [ ]:
import time
from IPython.display import display, Javascript
from sklearn.metrics import precision_score, recall_score

# Display the message "Runtime is still alive" every 30 minutes
# Send a request to maintain the runtime
display(Javascript('''
 function ClickConnect(){
   console.log("Working"); 
   document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,1000*60*30);
'''))
# Define the path where to save the checkpoint
checkpoint_path = '/content/drive/MyDrive/진짜 딥러닝 진행 세이브.pth'

# Training loop
num_epochs = 10
best_accuracy = 0.0
best_f1_score = 0.0 # 추가
best_precision = 0.0
best_recall = 0.0
for epoch in range(num_epochs):
    running_loss = 0.0
    
    # Training loop
    model.train() # Set model to training mode
    
    for i, (images, labels) in enumerate(train_dataloader):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Print progress every 10 batches
        if (i+1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{i+1}/{len(train_dataloader)}], Loss: {loss.item()}")

    epoch_loss = running_loss / len(train_dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_loss}")

    # Validation loop
    model.eval() # Set model to evaluation mode
    with torch.no_grad():
        correct = 0
        total = 0
        val_loss = 0.0
        y_true, y_pred = [], []
        for i, (images, labels) in enumerate(val_dataloader):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            # Print progress every 10 batches
        if (i+1) % 10 == 0:
            print(f"Validation: Epoch [{epoch+1}/{num_epochs}], Batch [{i+1}/{len(val_dataloader)}], Loss: {loss.item()}, Accuracy: {(predicted == labels).sum().item() / labels.size(0)}")
        accuracy = correct / total
        val_loss /= len(val_dataloader)
        f1 = f1_score(y_true, y_pred, average='weighted')
        precision = precision_score(y_true, y_pred, average='weighted')
        recall = recall_score(y_true, y_pred, average='weighted')
        print(f"Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss}, Accuracy: {accuracy}, F1-score: {f1}, Precision: {precision}, Recall: {recall}")

        
        # Save the best model checkpoint based on validation accuracy and F1-score
        if accuracy > best_accuracy or f1 > best_f1_score or precision > best_precision or recall > best_recall:
            if f1 > best_f1_score:
                print(f"F1-score improved from {best_f1_score:.4f} to {f1:.4f}")
                best_f1_score = f1
            if accuracy > best_accuracy:
                print(f"Accuracy improved from {best_accuracy:.4f} to {accuracy:.4f}")
                best_accuracy = accuracy
            if precision > best_precision:
                print(f"Precision improved from {best_precision:.4f} to {precision:.4f}")
            if recall > best_recall:
                print(f"Recall improved from {best_recall:.4f} to {recall:.4f}")
                best_recall = recall
    
    # Save the model checkpoint after each epoch
    checkpoint_path = f"music_classifier_epoch{epoch+1}.pth"
    torch.save(model.state_dict(), checkpoint_path)

print("Training finished.")

# Save the final trained model
checkpoint_path = os.path.join(checkpoint_dir, "music_classifier_final.pth")
torch.save(model.state_dict(), "music_classifier.pth")

In [ ]:

# Load the stored checkpoint
checkpoint_path = '/content/gdrive/MyDrive/music_classifier.pth'
model.load_state_dict(torch.load(checkpoint_path))